# Large-Scale Stochastic Variational GP Regression (CUDA) (w/ SVGP)

## Overview

In this notebook, we'll give an overview of how to use Deep Kernel Learning with SVGP stochastic variational regression to rapidly train using minibatches on the `3droad` UCI dataset with hundreds of thousands of training examples. 

In contrast to the SVDKL_Regression_GridInterp_CUDA notebook, we'll be using SVGP (https://arxiv.org/pdf/1411.2005.pdf) here to learn the inducing point locations. Our implementation of SVGP is modified to be efficient with the inference techniques used in GPyTorch.

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `song` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', '3droad.mat')
    
data = torch.Tensor(loadmat('3droad.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [3]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

## Defining the DKL Feature Extractor

Next, we define the neural network feature extractor used to define the deep kernel. In this case, we use a fully connected network with the architecture `d -> 1000 -> 500 -> 50 -> 2`, as described in the original DKL paper. All of the code below uses standard PyTorch implementations of neural network layers.

In [4]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):           
    def __init__(self):                                      
        super(LargeFeatureExtractor, self).__init__()        
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('bn1', torch.nn.BatchNorm1d(1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 1000))
        self.add_module('bn2', torch.nn.BatchNorm1d(1000))
        self.add_module('relu2', torch.nn.ReLU())                       
        self.add_module('linear3', torch.nn.Linear(1000, 500))
        self.add_module('bn3', torch.nn.BatchNorm1d(500))
        self.add_module('relu3', torch.nn.ReLU())                  
        self.add_module('linear4', torch.nn.Linear(500, 50))       
        self.add_module('bn4', torch.nn.BatchNorm1d(50))
        self.add_module('relu4', torch.nn.ReLU())                  
        self.add_module('linear5', torch.nn.Linear(50, 2))         
                                                             
feature_extractor = LargeFeatureExtractor().cuda()
# num_features is the number of final features extracted by the neural network, in this case 2.
num_features = 2

## Defining the GP Regression Layer

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. In this example, because we will be learning the inducing point locations, we'll be using a base `VariationalStrategy` with `learn_inducing_locations=True`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

In [5]:
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy
class GPRegressionLayer(AbstractVariationalGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPRegressionLayer, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(
            log_lengthscale_prior=gpytorch.priors.SmoothedBoxPrior(0.001, 1., sigma=0.1, log_transform=True)
        ))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

## Defining the DKL Model

With the feature extractor and GP regression layer defined, we can now define our full model. To do this, we simply create a module whose `forward()` method passes the data first through the feature extractor, and then through the GP regression layer.

The only other interesting feature of the model below is that we use a helper function, `scale_to_bounds`, to ensure that the features extracted by the neural network fit within the grid bounds used for SKI.

In [6]:
class DKLModel(gpytorch.Module):
    def __init__(self, inducing_points, feature_extractor, num_features, grid_bounds=(-1., 1.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GPRegressionLayer(inducing_points)
        self.grid_bounds = grid_bounds
        self.num_features = num_features

    def forward(self, x):
        features = self.feature_extractor(x)
        features = gpytorch.utils.grid.scale_to_bounds(features, self.grid_bounds[0], self.grid_bounds[1])
        res = self.gp_layer(features)
        return res
inducing_points = gpytorch.utils.grid.scale_to_bounds(feature_extractor(train_x[:500, :]), -1, 1)
model = DKLModel(inducing_points=inducing_points, feature_extractor=feature_extractor, num_features=num_features).cuda()
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

## Training the Model

The cell below trains the DKL model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalMarginalLogLikelihood` or ELBO), call backwards, and do a step of optimization.

In [7]:
model.train()
likelihood.train()

# We'll do 1 epochs of training in this tutorial
num_epochs = 6

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-3},
    {'params': model.gp_layer.parameters()},
    {'params': likelihood.parameters()},
], lr=0.1)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=train_y.size(0), combine_terms=False)

for i in range(num_epochs):
    scheduler.step()
    # Within each iteration, we will go over each minibatch of data
    for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):        
        optimizer.zero_grad()
        
        # Because the grid is relatively small, we turn off the Toeplitz matrix multiplication and just perform them directly
        # We find this to be more efficient when the grid is very small.
        with gpytorch.settings.use_toeplitz(False):
            output = model(x_batch)
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

        # The actual optimization step
        loss.backward()
        optimizer.step()

/home/jrg365/gpytorch/gpytorch/utils/cholesky.py:14: UserWarning: torch.potrf is deprecated in favour of torch.cholesky and will be removed in the next release. Please use torch.cholesky instead and note that the :attr:`upper` argument in torch.cholesky defaults to ``False``.
  potrf_list = [sub_mat.potrf() for sub_mat in mat.view(-1, *mat.shape[-2:])]
/home/jrg365/gpytorch/gpytorch/lazy/added_diag_lazy_tensor.py:74: UserWarning: torch.potrf is deprecated in favour of torch.cholesky and will be removed in the next release. Please use torch.cholesky instead and note that the :attr:`upper` argument in torch.cholesky defaults to ``False``.
  ld_one = lr_flipped.potrf().diag().log().sum() * 2


Epoch 1 [0/340] - Loss: 171.225 [-169.997, 1.229, -0.000]
Epoch 1 [1/340] - Loss: 167.302 [-166.596, 0.707, -0.000]
Epoch 1 [2/340] - Loss: 139.774 [-139.006, 0.768, -0.000]
Epoch 1 [3/340] - Loss: 125.479 [-124.646, 0.833, -0.000]
Epoch 1 [4/340] - Loss: 119.970 [-119.137, 0.833, -0.000]
Epoch 1 [5/340] - Loss: 114.708 [-113.955, 0.753, -0.000]
Epoch 1 [6/340] - Loss: 94.375 [-93.734, 0.641, -0.000]
Epoch 1 [7/340] - Loss: 87.867 [-87.327, 0.541, -0.000]
Epoch 1 [8/340] - Loss: 76.002 [-75.536, 0.466, -0.000]
Epoch 1 [9/340] - Loss: 68.272 [-67.854, 0.417, -0.000]
Epoch 1 [10/340] - Loss: 66.167 [-65.778, 0.389, -0.000]
Epoch 1 [11/340] - Loss: 63.866 [-63.465, 0.400, -0.000]
Epoch 1 [12/340] - Loss: 54.164 [-53.690, 0.474, -0.000]
Epoch 1 [13/340] - Loss: 52.675 [-52.107, 0.569, -0.000]
Epoch 1 [14/340] - Loss: 45.446 [-44.798, 0.649, -0.000]
Epoch 1 [15/340] - Loss: 40.557 [-39.868, 0.690, -0.000]
Epoch 1 [16/340] - Loss: 42.817 [-42.124, 0.693, -0.000]
Epoch 1 [17/340] - Loss: 41.0

Epoch 1 [146/340] - Loss: 6.269 [-6.153, 0.116, -0.000]
Epoch 1 [147/340] - Loss: 5.786 [-5.675, 0.111, -0.000]
Epoch 1 [148/340] - Loss: 6.006 [-5.895, 0.111, -0.000]
Epoch 1 [149/340] - Loss: 6.080 [-5.961, 0.119, -0.000]
Epoch 1 [150/340] - Loss: 6.965 [-6.844, 0.120, -0.000]
Epoch 1 [151/340] - Loss: 6.138 [-6.028, 0.111, -0.000]
Epoch 1 [152/340] - Loss: 6.168 [-6.078, 0.090, -0.000]
Epoch 1 [153/340] - Loss: 5.905 [-5.828, 0.076, -0.000]
Epoch 1 [154/340] - Loss: 5.825 [-5.755, 0.069, -0.000]
Epoch 1 [155/340] - Loss: 5.967 [-5.892, 0.076, -0.000]
Epoch 1 [156/340] - Loss: 9.614 [-9.537, 0.078, -0.000]
Epoch 1 [157/340] - Loss: 6.037 [-5.973, 0.064, -0.000]
Epoch 1 [158/340] - Loss: 6.340 [-6.290, 0.050, -0.000]
Epoch 1 [159/340] - Loss: 5.718 [-5.657, 0.060, -0.000]
Epoch 1 [160/340] - Loss: 5.786 [-5.728, 0.059, -0.000]
Epoch 1 [161/340] - Loss: 5.847 [-5.803, 0.044, -0.000]
Epoch 1 [162/340] - Loss: 5.724 [-5.683, 0.041, -0.000]
Epoch 1 [163/340] - Loss: 6.567 [-6.529, 0.038, 

Epoch 1 [293/340] - Loss: 4.236 [-4.231, 0.005, -0.000]
Epoch 1 [294/340] - Loss: 4.310 [-4.305, 0.005, -0.000]
Epoch 1 [295/340] - Loss: 4.278 [-4.272, 0.005, -0.000]
Epoch 1 [296/340] - Loss: 4.343 [-4.338, 0.005, -0.000]
Epoch 1 [297/340] - Loss: 4.299 [-4.293, 0.005, -0.000]
Epoch 1 [298/340] - Loss: 4.195 [-4.189, 0.005, -0.000]
Epoch 1 [299/340] - Loss: 4.158 [-4.152, 0.006, -0.000]
Epoch 1 [300/340] - Loss: 4.264 [-4.258, 0.006, -0.000]
Epoch 1 [301/340] - Loss: 4.255 [-4.249, 0.006, -0.000]
Epoch 1 [302/340] - Loss: 4.393 [-4.387, 0.006, -0.000]
Epoch 1 [303/340] - Loss: 4.290 [-4.283, 0.007, -0.000]
Epoch 1 [304/340] - Loss: 4.280 [-4.273, 0.007, -0.000]
Epoch 1 [305/340] - Loss: 4.312 [-4.305, 0.007, -0.000]
Epoch 1 [306/340] - Loss: 4.549 [-4.542, 0.007, -0.000]
Epoch 1 [307/340] - Loss: 4.442 [-4.435, 0.007, -0.000]
Epoch 1 [308/340] - Loss: 4.357 [-4.350, 0.006, -0.000]
Epoch 1 [309/340] - Loss: 4.431 [-4.425, 0.006, -0.000]
Epoch 1 [310/340] - Loss: 4.346 [-4.340, 0.006, 

Epoch 2 [103/340] - Loss: 4.137 [-4.134, 0.003, -0.000]
Epoch 2 [104/340] - Loss: 4.124 [-4.121, 0.003, -0.000]
Epoch 2 [105/340] - Loss: 4.191 [-4.189, 0.003, -0.000]
Epoch 2 [106/340] - Loss: 4.145 [-4.142, 0.003, -0.000]
Epoch 2 [107/340] - Loss: 4.137 [-4.134, 0.003, -0.000]
Epoch 2 [108/340] - Loss: 4.210 [-4.208, 0.003, -0.000]
Epoch 2 [109/340] - Loss: 4.133 [-4.130, 0.003, -0.000]
Epoch 2 [110/340] - Loss: 4.182 [-4.179, 0.003, -0.000]
Epoch 2 [111/340] - Loss: 4.194 [-4.191, 0.003, -0.000]
Epoch 2 [112/340] - Loss: 4.153 [-4.150, 0.003, -0.000]
Epoch 2 [113/340] - Loss: 4.053 [-4.050, 0.003, -0.000]
Epoch 2 [114/340] - Loss: 4.145 [-4.141, 0.003, -0.000]
Epoch 2 [115/340] - Loss: 4.132 [-4.128, 0.003, -0.000]
Epoch 2 [116/340] - Loss: 4.158 [-4.155, 0.004, -0.000]
Epoch 2 [117/340] - Loss: 4.135 [-4.131, 0.004, -0.000]
Epoch 2 [118/340] - Loss: 4.151 [-4.148, 0.004, -0.000]
Epoch 2 [119/340] - Loss: 4.074 [-4.070, 0.003, -0.000]
Epoch 2 [120/340] - Loss: 4.203 [-4.199, 0.004, 

...

Epoch 4 [313/340] - Loss: 4.008 [-4.006, 0.002, -0.000]
Epoch 4 [314/340] - Loss: 3.945 [-3.944, 0.002, -0.000]
Epoch 4 [315/340] - Loss: 3.992 [-3.990, 0.002, -0.000]
Epoch 4 [316/340] - Loss: 3.980 [-3.978, 0.002, -0.000]
Epoch 4 [317/340] - Loss: 3.984 [-3.982, 0.002, -0.000]
Epoch 4 [318/340] - Loss: 3.996 [-3.994, 0.002, -0.000]
Epoch 4 [319/340] - Loss: 3.944 [-3.942, 0.002, -0.000]
Epoch 4 [320/340] - Loss: 3.956 [-3.955, 0.002, -0.000]
Epoch 4 [321/340] - Loss: 3.952 [-3.950, 0.002, -0.000]
Epoch 4 [322/340] - Loss: 3.950 [-3.948, 0.002, -0.000]
Epoch 4 [323/340] - Loss: 4.065 [-4.063, 0.002, -0.000]
Epoch 4 [324/340] - Loss: 3.947 [-3.945, 0.002, -0.000]
Epoch 4 [325/340] - Loss: 4.076 [-4.074, 0.002, -0.000]
Epoch 4 [326/340] - Loss: 4.021 [-4.019, 0.002, -0.000]
Epoch 4 [327/340] - Loss: 3.995 [-3.993, 0.002, -0.000]
Epoch 4 [328/340] - Loss: 3.962 [-3.960, 0.002, -0.000]
Epoch 4 [329/340] - Loss: 3.986 [-3.984, 0.002, -0.000]
Epoch 4 [330/340] - Loss: 3.949 [-3.947, 0.002, 

Epoch 5 [123/340] - Loss: 3.984 [-3.982, 0.002, -0.000]
Epoch 5 [124/340] - Loss: 3.960 [-3.958, 0.002, -0.000]
Epoch 5 [125/340] - Loss: 4.012 [-4.010, 0.002, -0.000]
Epoch 5 [126/340] - Loss: 3.952 [-3.950, 0.002, -0.000]
Epoch 5 [127/340] - Loss: 3.971 [-3.969, 0.002, -0.000]
Epoch 5 [128/340] - Loss: 3.967 [-3.965, 0.002, -0.000]
Epoch 5 [129/340] - Loss: 4.064 [-4.062, 0.002, -0.000]
Epoch 5 [130/340] - Loss: 4.002 [-4.000, 0.002, -0.000]
Epoch 5 [131/340] - Loss: 3.992 [-3.989, 0.002, -0.000]
Epoch 5 [132/340] - Loss: 4.038 [-4.036, 0.002, -0.000]
Epoch 5 [133/340] - Loss: 3.967 [-3.965, 0.002, -0.000]
Epoch 5 [134/340] - Loss: 3.968 [-3.966, 0.002, -0.000]
Epoch 5 [135/340] - Loss: 3.986 [-3.984, 0.002, -0.000]
Epoch 5 [136/340] - Loss: 3.949 [-3.947, 0.002, -0.000]
Epoch 5 [137/340] - Loss: 3.997 [-3.995, 0.002, -0.000]
Epoch 5 [138/340] - Loss: 3.963 [-3.961, 0.002, -0.000]
Epoch 5 [139/340] - Loss: 4.031 [-4.029, 0.002, -0.000]
Epoch 5 [140/340] - Loss: 3.950 [-3.948, 0.002, 

Epoch 5 [270/340] - Loss: 3.965 [-3.963, 0.002, -0.000]
Epoch 5 [271/340] - Loss: 3.958 [-3.956, 0.002, -0.000]
Epoch 5 [272/340] - Loss: 3.952 [-3.950, 0.002, -0.000]
Epoch 5 [273/340] - Loss: 4.015 [-4.013, 0.002, -0.000]
Epoch 5 [274/340] - Loss: 3.958 [-3.956, 0.002, -0.000]
Epoch 5 [275/340] - Loss: 3.933 [-3.931, 0.002, -0.000]
Epoch 5 [276/340] - Loss: 3.922 [-3.920, 0.002, -0.000]
Epoch 5 [277/340] - Loss: 3.965 [-3.963, 0.002, -0.000]
Epoch 5 [278/340] - Loss: 3.912 [-3.910, 0.002, -0.000]
Epoch 5 [279/340] - Loss: 4.021 [-4.019, 0.002, -0.000]
Epoch 5 [280/340] - Loss: 3.964 [-3.962, 0.002, -0.000]
Epoch 5 [281/340] - Loss: 3.988 [-3.986, 0.002, -0.000]
Epoch 5 [282/340] - Loss: 4.001 [-3.999, 0.002, -0.000]
Epoch 5 [283/340] - Loss: 4.015 [-4.014, 0.002, -0.000]
Epoch 5 [284/340] - Loss: 3.989 [-3.987, 0.002, -0.000]
Epoch 5 [285/340] - Loss: 3.965 [-3.964, 0.002, -0.000]
Epoch 5 [286/340] - Loss: 3.930 [-3.928, 0.002, -0.000]
Epoch 5 [287/340] - Loss: 4.021 [-4.020, 0.002, 

Epoch 6 [80/340] - Loss: 3.934 [-3.932, 0.002, -0.000]
Epoch 6 [81/340] - Loss: 3.965 [-3.963, 0.002, -0.000]
Epoch 6 [82/340] - Loss: 3.926 [-3.924, 0.002, -0.000]
Epoch 6 [83/340] - Loss: 3.917 [-3.915, 0.002, -0.000]
Epoch 6 [84/340] - Loss: 3.923 [-3.921, 0.002, -0.000]
Epoch 6 [85/340] - Loss: 3.963 [-3.960, 0.002, -0.000]
Epoch 6 [86/340] - Loss: 3.962 [-3.959, 0.002, -0.000]
Epoch 6 [87/340] - Loss: 3.970 [-3.968, 0.002, -0.000]
Epoch 6 [88/340] - Loss: 3.970 [-3.968, 0.002, -0.000]
Epoch 6 [89/340] - Loss: 3.935 [-3.933, 0.002, -0.000]
Epoch 6 [90/340] - Loss: 3.897 [-3.895, 0.002, -0.000]
Epoch 6 [91/340] - Loss: 4.040 [-4.038, 0.002, -0.000]
Epoch 6 [92/340] - Loss: 4.012 [-4.010, 0.002, -0.000]
Epoch 6 [93/340] - Loss: 3.989 [-3.987, 0.002, -0.000]
Epoch 6 [94/340] - Loss: 3.968 [-3.966, 0.002, -0.000]
Epoch 6 [95/340] - Loss: 3.924 [-3.922, 0.002, -0.000]
Epoch 6 [96/340] - Loss: 3.946 [-3.944, 0.002, -0.000]
Epoch 6 [97/340] - Loss: 3.947 [-3.945, 0.002, -0.000]
Epoch 6 [9

Epoch 6 [228/340] - Loss: 3.927 [-3.925, 0.002, -0.000]
Epoch 6 [229/340] - Loss: 3.966 [-3.964, 0.002, -0.000]
Epoch 6 [230/340] - Loss: 3.987 [-3.985, 0.002, -0.000]
Epoch 6 [231/340] - Loss: 3.958 [-3.956, 0.002, -0.000]
Epoch 6 [232/340] - Loss: 3.940 [-3.937, 0.002, -0.000]
Epoch 6 [233/340] - Loss: 3.965 [-3.963, 0.002, -0.000]
Epoch 6 [234/340] - Loss: 3.945 [-3.943, 0.002, -0.000]
Epoch 6 [235/340] - Loss: 3.953 [-3.951, 0.002, -0.000]
Epoch 6 [236/340] - Loss: 3.949 [-3.946, 0.002, -0.000]
Epoch 6 [237/340] - Loss: 3.967 [-3.965, 0.002, -0.000]
Epoch 6 [238/340] - Loss: 3.925 [-3.923, 0.002, -0.000]
Epoch 6 [239/340] - Loss: 3.974 [-3.972, 0.002, -0.000]
Epoch 6 [240/340] - Loss: 3.947 [-3.945, 0.002, -0.000]
Epoch 6 [241/340] - Loss: 3.975 [-3.973, 0.002, -0.000]
Epoch 6 [242/340] - Loss: 3.901 [-3.899, 0.002, -0.000]
Epoch 6 [243/340] - Loss: 3.914 [-3.912, 0.002, -0.000]
Epoch 6 [244/340] - Loss: 3.937 [-3.935, 0.002, -0.000]
Epoch 6 [245/340] - Loss: 3.949 [-3.946, 0.002, 

## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`) using the standard SKI testing code, with no acceleration or precomputation. Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although our other tutorials demonstrate how to do this (for example, see the CIFAR tutorial).

In [8]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False):
    preds = model(test_x)

/home/jrg365/gpytorch/gpytorch/utils/cholesky.py:14: UserWarning: torch.potrf is deprecated in favour of torch.cholesky and will be removed in the next release. Please use torch.cholesky instead and note that the :attr:`upper` argument in torch.cholesky defaults to ``False``.
  potrf_list = [sub_mat.potrf() for sub_mat in mat.view(-1, *mat.shape[-2:])]
/home/jrg365/gpytorch/gpytorch/lazy/added_diag_lazy_tensor.py:74: UserWarning: torch.potrf is deprecated in favour of torch.cholesky and will be removed in the next release. Please use torch.cholesky instead and note that the :attr:`upper` argument in torch.cholesky defaults to ``False``.
  ld_one = lr_flipped.potrf().diag().log().sum() * 2


In [9]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 8.996102333068848
